# Text to SQL 테스트

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
user_msg = """### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### Make a query to list the names of the departments which employed more than 10 employees in the last 3 months.
query:
"""

# user_msg = """### Postgres SQL tables, with their properties:
# #
# # Employee(id, name, department_id)
# # Department(id, name, address)
# # Salary_Payments(id, employee_id, amount, date)
# #
# ### 최근 3 개월간 직원을 10명 이상 뽑은 부서의 이름을 수집하는 쿼리를 만들어줘.
# query: 
# """

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

To list the names of the departments that employed more than 10 employees in the last 3 months, you can use the following SQL query. This query joins the `Employee`, `Department`, and `Salary_Payments` tables, filters the salary payments to the last 3 months, groups by department, and counts the employees.

```sql
SELECT d.name AS department_name
FROM Department d
JOIN Employee e ON d.id = e.department_id
JOIN Salary_Payments sp ON e.id = sp.employee_id
WHERE sp.date >= NOW() - INTERVAL '3 months'
GROUP BY d.id, d.name
HAVING COUNT(DISTINCT e.id) > 10;
```

### Explanation:
1. **JOINs**: The query joins the `Department`, `Employee`, and `Salary_Payments` tables to link employees to their respective departments and their salary payments.
2. **WHERE clause**: It filters the salary payments to only include those made in the last 3 months using `NOW() - INTERVAL '3 months'`.
3. **GROUP BY**: It groups the results by department ID and name to aggregate the employee counts.
4. **HAVING claus

# SQL to Text 테스트

In [3]:
user_msg = """###Can you explain what does this code do? 
# Code: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# Answer: 
"""

# user_msg = """###이 코드가 무엇을 수행하는지 설명해줘?
# #Code:
# SELECT d.name
# FROM Department d
# WHERE d.id IN (
#     SELECT e.department_id
#     FROM Employee e
#     WHERE e.id IN (
#         SELECT sp.employee_id
#         FROM Salary_Payments sp
#         WHERE sp.date >= current_date - interval '3 months'
#     )
#     GROUP BY e.department_id
#     HAVING COUNT(e.id) > 10
# )
# Answer:
# """
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

This SQL code is designed to retrieve the names of departments that have a specific criterion based on employee activity over the last three months. Let's break down the code step by step:

1. **Outer Query**: 
   ```sql
   SELECT d.name
   FROM Department d
   WHERE d.id IN (...)
   ```
   - This part of the query selects the names of departments (`d.name`) from the `Department` table (`Department d`). 
   - The `WHERE` clause filters the departments to include only those that have an `id` present in the results returned by the subquery in the parentheses.

2. **First Inner Subquery**:
   ```sql
   SELECT e.department_id
   FROM Employee e
   WHERE e.id IN (...)
   GROUP BY e.department_id
   HAVING COUNT(e.id) > 10
   ```
   - This subquery selects `department_id` from the `Employee` table (`Employee e`).
   - It filters the employee records to include only those employees whose IDs are present in the results of another subquery (nested subquery).
   - After filtering employees, it g